In [1]:
import os
import torch
import numpy as np
import pandas as pd
from CellClass import CNN
from natsort import natsorted
import matplotlib.pyplot as plt
from CellClass.CNN import dataset
import CellClass.CNN.dataset as D
import CellClass.CNN.training as T
from torch.utils.data import DataLoader

In [2]:
save_dir = "/home/simon_g/src/MICCAI/trained_models"

setups = os.listdir(save_dir)

setup_best = []
for setup in setups:
    models = [os.path.join(save_dir, setup, x) for x in os.listdir(os.path.join(save_dir, setup)) if ".pt" in x]
    
    best = np.inf
    for model in models:
        dict_mod = torch.load(model)
        if dict_mod["validation_loss"] < best:
            best = dict_mod["validation_loss"]
            best_model = model
        
    setup_best.append([setup, best_model])
    
setup_best = np.array(setup_best)

In [3]:
# try:
#     layers = np.array(setup_df.layers.item().strip().replace("[", "").replace("]", "").split(",")).astype(int)
#     in_shape = np.array(setup_df.in_shape.item().strip().replace("[", "").replace("]", "").split(",")).astype(int)

# except:
layers = np.array([3, 16, 64, 128, 256])
in_shape = np.array([128, 128])

In [4]:
patches_dir = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/MYCN_SpikeIn/results/kept"
dilutions = np.unique([x.split("_")[0] + "_" for x in os.listdir(patches_dir)])

dils = []
all_files = os.listdir(patches_dir)
for dilution in dilutions: 
    files = [x for x in all_files if f"{dilution}" in x]
    dils.append({"series": dilution, "files": files})
    print(dilution, len(files))
    
original_dilutions = [{
    "S1b_": 95, "S2_": 75, "S3_": 50,
    "S4_": 100, "S6_": 0, "S7_": 90,
    "S8_": 25, "S11_": 99, "S12_": 0,
    "S19_": 100, "S20_": 99, "S29_": 0,
    "S30_": 95, "S31_": 50, "S32_": 90,
    "S33_": 25, "S34_": 75,
}]

print(original_dilutions)

S11_ 980
S12_ 885
S19_ 948
S1b_ 927
S20_ 980
S29_ 950
S2_ 964
S30_ 973
S31_ 974
S32_ 968
S33_ 860
S34_ 957
S3_ 861
S4_ 979
S6_ 752
S7_ 970
S8_ 955
[{'S1b_': 95, 'S2_': 75, 'S3_': 50, 'S4_': 100, 'S6_': 0, 'S7_': 90, 'S8_': 25, 'S11_': 99, 'S12_': 0, 'S19_': 100, 'S20_': 99, 'S29_': 0, 'S30_': 95, 'S31_': 50, 'S32_': 90, 'S33_': 25, 'S34_': 75}]


In [7]:
import importlib
importlib.reload(dataset)
importlib.reload(T)
import random
from torchvision import transforms

all_results = []
for (setup, best_model), rescale in zip(setup_best, [True, False]):
    print(setup, best_model, rescale)

    best = torch.load(best_model, map_location="cpu")
    model = CNN.ClassificationCNN()
    model.load_state_dict(best["model_state_dict"])

    dilutions = natsorted(dilutions)
    results = {}
    for dil in dilutions:
        files = [os.path.join(patches_dir, x) for x in os.listdir(patches_dir) if f"{dil}" in x]
        random.shuffle(files)
        dilution = dataset.PatchDataset(files[:1000], eval=True, rescale_intensity=rescale)
        dilution_loader = DataLoader(dilution, 128, num_workers=2)
        ims, labels, percentage = T.predict_dilution(model, dilution_loader, with_uncertainty=True)
        results[dil] = percentage
            
        print(f"{dil} [{len(dilution)} Patches] with predicted dilution of: {percentage}")
    all_results.append(results)

20220629_135930 /home/simon_g/src/MICCAI/trained_models/20220629_135930/CNN_Model_20220629_212254814.pt True
S1b_ [927 Patches] with predicted dilution of: 95.69
S2_ [964 Patches] with predicted dilution of: 96.23
S3_ [861 Patches] with predicted dilution of: 66.86
S4_ [979 Patches] with predicted dilution of: 99.9
S6_ [752 Patches] with predicted dilution of: 4.26
S7_ [970 Patches] with predicted dilution of: 97.09
S8_ [955 Patches] with predicted dilution of: 29.12
S11_ [980 Patches] with predicted dilution of: 99.68
S12_ [885 Patches] with predicted dilution of: 3.2
S19_ [948 Patches] with predicted dilution of: 99.68
S20_ [980 Patches] with predicted dilution of: 98.83
S29_ [950 Patches] with predicted dilution of: 0.11
S30_ [973 Patches] with predicted dilution of: 96.59
S31_ [974 Patches] with predicted dilution of: 49.15
S32_ [968 Patches] with predicted dilution of: 94.63
S33_ [860 Patches] with predicted dilution of: 1.0
S34_ [957 Patches] with predicted dilution of: 79.3
2022

In [8]:
df_target = pd.DataFrame(original_dilutions)
df_prediction = pd.DataFrame(all_results)

merged = pd.concat((df_target, df_prediction), axis=0)
rescaled = {}
nrescaled = {}
for i,j in merged.items():
   rescaled[i] = j.iloc[1]-j.iloc[0]
   nrescaled[i] = j.iloc[2]-j.iloc[0]

   
merged = pd.concat((merged, pd.DataFrame([rescaled, nrescaled])), axis=0)
merged.index = ["original", "rescales", "not rescaled", "diff rescaled", "diff not rescaled"]
merged.head()

,S1b_,S2_,S3_,S4_,S6_,S7_,S8_,S11_,S12_,S19_,S20_,S29_,S30_,S31_,S32_,S33_,S34_
original,95.00,75.00,50.00,100.00,0.00,90.00,25.00,99.00,0.00,100.00,99.00,0.00,95.00,50.00,90.00,25.00,75.0
rescales,95.69,96.23,66.86,99.90,4.26,97.09,29.12,99.68,3.20,99.68,98.83,0.11,96.59,49.15,94.63,1.00,79.3
not rescaled,87.71,81.94,59.70,96.68,0.33,91.85,18.41,95.49,0.11,99.68,98.10,0.11,96.25,49.66,94.26,7.12,75.8
diff rescaled,0.69,21.23,16.86,-0.10,4.26,7.09,4.12,0.68,3.20,-0.32,-0.17,0.11,1.59,-0.85,4.63,-24.00,4.3
diff not rescaled,-7.29,6.94,9.70,-3.32,0.33,1.85,-6.59,-3.51,0.11,-0.32,-0.90,0.11,1.25,-0.34,4.26,-17.88,0.8


In [ ]:
from CellClass import utils
from CellClass.CNN import transformations
import importlib
importlib.reload(transformations)
importlib.reload(utils)
out = "/home/simon_g/src/MICCAI/predicted_samples"
patches_dir = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/MYCN_SpikeIn/results/kept"
dilutions = np.unique([x.split("_")[0] + "_" for x in os.listdir(patches_dir)])

best = torch.load(setup_best[0][1], map_location="cpu")
model = CNN.ClassificationCNN()
model.load_state_dict(best["model_state_dict"])

for dil in dilutions:
    
    files = [os.path.join(patches_dir, x) for x in os.listdir(patches_dir) if dil in x]
    random.shuffle(files)
    filenames = [x.split("/")[-1] for x in files]
    dilution = dataset.PatchDataset(files[:1000], eval=True, rescale_intensity=True)
    dilution_loader = DataLoader(dilution, 128, num_workers=2)
    ims, labels, percentage = T.predict_dilution(model, dilution_loader)
    
    filenames = np.array(filenames)
    labels = np.array(labels).squeeze()
    pos_idxs = np.where(labels==1)
    neg_idxs = np.where(labels==0)
    pos = filenames[pos_idxs]
    neg = filenames[neg_idxs]
    ims = np.array(ims)
    ims = torch.tensor(ims).permute(0,3,1,2)
    t = transformations.DeNormalize(rescaled=True)
    ims = t(ims).permute(0,2,3,1).detach().numpy()
    utils.gridPlot(ims[pos_idxs], labels=pos, save=os.path.join(out, f"{dil}_pos_rescaled.png"), plot=False)
    utils.gridPlot(ims[neg_idxs], labels=neg, save=os.path.join(out, f"{dil}_neg_rescaled.png"), plot=False)
